In [ ]:
# Original installations with minor cleanup and ALLaM compatibility
!pip install transformers datasets torch accelerate bitsandbytes wandb arabic-reshaper python-bidi
!pip install git+https://github.com/MagedSaeed/Bohour.git
!pip install -U transformers sentencepiece accelerate datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import re
from collections import Counter

import torch
import arabic_reshaper
from bidi.algorithm import get_display
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments, Trainer
)
from peft import (
    LoraConfig, get_peft_model,
    prepare_model_for_kbit_training, TaskType
)



In [ ]:
import collections
import re
from datasets import load_dataset

# Constants for tokenization
ST_BAYT_TOKEN = "<|bayt|>"
VERSE_TOKEN = "<|verse|>"
ED_BAYT_TOKEN = "<|endbayt|>"

meter_tokens = {
    "الطويل": "<|meter_1|>",
    "البسيط": "<|meter_2|>",
    "الكامل": "<|meter_3|>",
    "الوافر": "<|meter_4|>",
    "السريع": "<|meter_5|>",
    "الخفيف": "<|meter_6|>"
}

theme_tokens = {
    "مدح": "<|theme_1|>",
    "رثاء": "<|theme_2|>",
    "غزل": "<|theme_3|>",
    "وصف": "<|theme_4|>",
    "حكمة": "<|theme_5|>",
    "فخر": "<|theme_6|>",
    "هجاء": "<|theme_7|>",
    "حماسة": "<|theme_8|>",
    "عتاب": "<|theme_9|>",
    "زهد": "<|theme_10|>",
    None: "<|theme_11|>"  # Default for missing themes
}

DIAC_RE = re.compile(r"[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]")
def strip_harakat(t): return DIAC_RE.sub("", t or "")

MAP = str.maketrans({"إ":"ا","أ":"ا","آ":"ا","ٱ":"ا","ى":"ي","ة":"ه"})
def norm(t): return strip_harakat(t).translate(MAP).strip()

def extract_qafiyah(verse, m=2):
    verse = norm(verse)
    verse = "".join(ch if "\u0621" <= ch <= "\u064A" else " " for ch in verse)
    last = verse.split()[-1] if verse.split() else ""
    rh = ""
    for ch in reversed(last):
        rh = ch+rh
        if len(rh) >= m: break
    return rh or None

def filter_by_qafiyah(sample):

    verses = sample['poem verses']
    if len(verses) < 2:
        return False

    qafiyahs = []
    for i in range(1, len(verses), 2):
        if i < len(verses):
            qafiyah = extract_qafiyah(verses[i])
            if qafiyah:
                qafiyahs.append(qafiyah)

    if not qafiyahs or len(qafiyahs) < 2:
        return False

    counter = collections.Counter(qafiyahs)
    most_common = counter.most_common(1)[0]
    consistency_ratio = most_common[1] / len(qafiyahs)

    return consistency_ratio >= 0.3

def get_qafiyah_majority(poem):

    if not poem or ED_BAYT_TOKEN not in poem or VERSE_TOKEN not in poem:
        return None

    qafiyahs = []
    for bayt in poem.split(ED_BAYT_TOKEN):
        if VERSE_TOKEN in bayt:
            parts = bayt.split(VERSE_TOKEN)
            if len(parts) >= 2:
                second_half = parts[1].strip()
                qafiyah = extract_qafiyah(second_half)
                if qafiyah:
                    qafiyahs.append(qafiyah)

    if not qafiyahs:
        return None

    counter = collections.Counter(qafiyahs)
    if not counter:
        return None

    most_common = counter.most_common(1)[0]
    dominant_qafiyah, count = most_common

    if count / len(qafiyahs) >= 0.5:
        return dominant_qafiyah

    return None

def load_dataset_and_preprocess():

    print("Loading the ashaar dataset...")
    ashaar = load_dataset("arbml/ashaar")

    print(f"Dataset loaded with {len(ashaar['train'])} entries")

    selected_meters = ["الخفيف", "الطويل", "الكامل", "البسيط", "السريع", "الوافر"]

    def process_verse(sample):

        chars = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويىئءأؤة ى'
        diacs = 'ْ~ًٌٍَُِّ'
        map_chars = {'ک': 'ك', 'ﺑ': 'ب', 'ٹ': 'ث', 'ی': 'ى'}
        out = []
        for verse in sample['poem verses']:
            proc_verse = ''
            for char in verse:
                if char in chars + diacs:
                    proc_verse += char
                elif char in map_chars:
                    proc_verse += map_chars[char]
            out.append(proc_verse)
        sample['poem verses'] = out
        return sample

    def filter_poems(sample):

        poem = sample['poem verses']
        if len(poem) < 2 or len(poem) % 2 != 0:
            return False
        return all(len(verse) >= 5 for verse in poem)

    def map_meters(sample):

        meter = sample['poem meter']
        if meter:
            if meter == 'بسيط':
                sample['poem meter'] = 'البسيط'
            elif 'خفيف' in meter:
                sample['poem meter'] = 'الخفيف'
            elif 'طويل' in meter:
                sample['poem meter'] = 'الطويل'
            elif 'كامل' in meter:
                sample['poem meter'] = 'الكامل'
            elif 'سريع' in meter:
                sample['poem meter'] = 'السريع'
            elif 'وافر' in meter:
                sample['poem meter'] = 'الوافر'
        return sample

    def filter_meters(sample):

        return sample['poem meter'] in selected_meters

    def join_verses(sample):

        verses = sample['poem verses']
        meter = sample['poem meter']
        theme = sample['poem theme']
        title = sample.get('poem title', 'بدون عنوان')





        poem = ''.join([f'{ST_BAYT_TOKEN} {verses[i]} {VERSE_TOKEN} {verses[i+1]} {ED_BAYT_TOKEN} '
                        for i in range(0, len(verses) - 1, 2)])

        qafiyah = get_qafiyah_majority(poem)
        if not qafiyah:
            return {"prompt": "", "completion": ""}

        prompt = f"""أنشئ قصيدة عربية فصيحة وفقاً للمواصفات التالية:

العنوان: {title}
البحر: {meter}
نوع القصيدة: {theme if theme else 'عامة'}
القافية: {qafiyah}

يجب أن تكون القصيدة:
- ملتزمة بقواعد بحر {meter} وتفعيلاته
- منتهية كل بيت بحرف :{qafiyah}
- متناسبة مع موضوع :{title}

الهيكل المطلوب:
{ST_BAYT_TOKEN} الشطر الأول {VERSE_TOKEN} الشطر الثاني {ED_BAYT_TOKEN}

اكتب القصيدة:"""

        completion = poem.strip()

        return {"prompt": prompt, "completion": completion}

    print("Processing verses...")
    ashaar = ashaar.map(process_verse)
    print("Filtering poems by structure...")
    ashaar = ashaar.filter(filter_poems)
    print("Mapping meters...")
    ashaar = ashaar.map(map_meters)
    print("Filtering by meter...")
    ashaar = ashaar.filter(filter_meters)
    print("Filtering by qafiyah...")
    ashaar = ashaar.filter(filter_by_qafiyah)
    print("Creating prompt-completion pairs...")
    processed_data = ashaar.map(join_verses)
    processed_data = processed_data.filter(lambda x: x["prompt"] != "" and x["completion"] != "")

    print(f"Final dataset size: {len(processed_data['train'])} entries")
    return processed_data

In [ ]:
processed_dataset = load_dataset_and_preprocess()


Loading the ashaar dataset...


dataset_infos.json:   0%|          | 0.00/34.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/126M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/151M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/254630 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/107357 [00:00<?, ? examples/s]

Creating prompt-completion pairs...


Map:   0%|          | 0/86164 [00:00<?, ? examples/s]

Filter:   0%|          | 0/86164 [00:00<?, ? examples/s]

Final dataset size: 77873 entries


In [ ]:
from pprint import pprint
pprint(processed_dataset['train']['prompt'][100])
pprint(processed_dataset['train']['completion'][100])

('أنشئ قصيدة عربية فصيحة وفقاً للمواصفات التالية:\n'
 '\n'
 'العنوان: لم تمل بي عن العفاف العقار\n'
 'البحر: الخفيف\n'
 'نوع القصيدة: قصيدة قصيره\n'
 'القافية: ار\n'
 '\n'
 'يجب أن تكون القصيدة:\n'
 '- ملتزمة بقواعد بحر الخفيف وتفعيلاته\n'
 '- منتهية كل بيت بحرف :ار\n'
 '- متناسبة مع موضوع :لم تمل بي عن العفاف العقار\n'
 '\n'
 'الهيكل المطلوب:\n'
 '<|bayt|> الشطر الأول <|verse|> الشطر الثاني <|endbayt|>\n'
 '\n'
 'اكتب القصيدة:')
('<|bayt|> لَم تَمل بي عَن العفاف العقارُ <|verse|> أَعشَق الغيد وَالوَقار '
 'وَقارُ <|endbayt|> <|bayt|> أَنظُم الشعر ما حييت وَأَني <|verse|> لابن بَيت '
 'تَهدى لَهُ الأَشعارُ <|endbayt|> <|bayt|> يَتَحلى بِيَ الزَمان تَحلى الغُص '
 '<|verse|> نِ لَما يَزينهُ النوّارُ <|endbayt|> <|bayt|> صَقَلتَني يَد '
 'التَجارُب حَتّى <|verse|> صَحَ عَزمي وَطابَ مِنهُ الغرارُ <|endbayt|> '
 '<|bayt|> وَمَكاني مِن الفَخار مَكان <|verse|> حَسدتهُ الشُموس وَالأَقمارُ '
 '<|endbayt|>')


In [ ]:
def load_model():
    model_name = "silma-ai/SILMA-9B-Instruct-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model with BF16 precision (ALLaM’s recommendation)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,  # Changed from float16 to match ALLaM’s native precision
        device_map="auto",
        load_in_8bit=True  # Kept for memory efficiency
    )

    # Ensure padding token exists (ALLaM may not define it)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "to_out.0"]
    )

    # Apply LoRA adapters
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    return model, tokenizer

model, tokenizer = load_model()

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

trainable params: 12,730,368 || all params: 9,254,436,352 || trainable%: 0.1376


In [ ]:
class PoetryDataset(torch.utils.data.Dataset):
    def __init__(self, examples, tokenizer, max_length=512):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        prompt = example["prompt"]
        completion = example["completion"]

        # Format as instruction tuning (unchanged)
        full_text = f"<s>[INST] {prompt} [/INST] {completion}</s>"

        # Tokenize
        encodings = self.tokenizer(
            full_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encodings["input_ids"][0]
        attention_mask = encodings["attention_mask"][0]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone()
        }

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the save path in Drive (updated for ALLaM)
save_path = "/content/drive/MyDrive/Silma_final_v3"

# Create the folder if it doesn’t exist
os.makedirs(save_path, exist_ok=True)

Mounted at /content/drive


In [ ]:
def setup_training(processed_data, tokenizer):
    train_dataset = PoetryDataset(
        processed_data["train"],
        tokenizer,
        max_length=512
    )

    training_args = TrainingArguments(
        output_dir=save_path,
        num_train_epochs=2,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
        learning_rate=2e-5,
        warmup_steps=500,
        bf16=True,  # Changed to match ALLaM’s precision
        fp16=False,  # Disabled since bf16 is used
        report_to="wandb",
        logging_dir=os.path.join(save_path, "logs"),
        dataloader_num_workers=2,
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    return train_dataset, training_args, data_collator

train_dataset, training_args, data_collator = setup_training(processed_dataset, tokenizer)

In [ ]:
def train_model(model, train_dataset, training_args, data_collator):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )

    trainer.train()

    # Updated save path for ALLaM within Google Drive
    save_path_drive = "/content/drive/MyDrive/Silma_final_v3"  # Organize within a folder
    os.makedirs(save_path_drive, exist_ok=True)
    model.save_pretrained(save_path_drive)
    tokenizer.save_pretrained(save_path_drive)

    return trainer

trainer = train_model(model, train_dataset, training_args, data_collator)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hatimalhomid (hatimalhomid-education-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs 

Step,Training Loss
100,2.993300
200,2.340200
300,1.654700
400,1.588400
500,1.593600
600,1.554200
700,1.520900
800,1.510500
900,1.501400
1000,1.494000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to 

Step,Training Loss
100,2.993300
200,2.340200
300,1.654700
400,1.588400
500,1.593600
600,1.554200
700,1.520900
800,1.510500
900,1.501400
1000,1.494000
